# Imports

In [ ]:
import pandas as pd
from nn_spikes import NeuralNetwork, batchTrain, test
import spike_tools, utilities

In [ ]:
data = pd.read_csv('./datasources/spikes/training_data.csv')
spikeLocations = pd.read_csv('./datasources/spikes/training_spike_locations.csv', index_col=0)

In [ ]:
data_training, data_validation, spikeIndexes_training, spikeIndexes_validation = spike_tools.dataPreProcess(data, spikeLocations)

# Run Neural Network Classifier

In [ ]:
results = utilities.createResultsRepo()

In [ ]:
nn = NeuralNetwork(input_nodes=101, hidden_nodes=int(hid), output_nodes=4, lr=0.1,error_function='difference-squared')

nn, trainingCurve, validationCurve = batchTrain(data_training=data_training,
                                                data_validation=data_validation,
                                                spikeIndexes_training=spikeIndexes_training, 
                                                spikeIndexes_validation=spikeIndexes_validation, 
                                                nn=nn,
                                                epochs=30,
                                                plotCurves=False)

In [ ]:
waveforms = data_validation[data_validation['predictedSpike']==True]['waveform']
predictionsMLP = classifySpikes(waveforms, nn)

# Run KNN Classifier

In [ ]:
predictionsKNN, componentsTraining = KNN_classifier(data_training.loc[spikeIndexes_training, 'waveform'].to_list(), 
                                                 data_validation.loc[spikeIndexes_validation, 'waveform'].to_list(), 
                                                 data_training.loc[spikeIndexes_training, 'knownClass'].to_list())

In [ ]:
px.scatter(x=componentsTraining.T[0], y=componentsTraining.T[1])

In [ ]:
matches = sum(data.loc[spikeIndexes_validation, 'knownClass'] == data.loc[spikeIndexes_validation, 'predictedClass'])

In [ ]:
acc = matches/len(data.loc[spikeIndexes_validation])
acc